In [1]:
############
# Tensorflow MAGIC
############
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [2]:
from __future__ import print_function
import numpy as np
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Simple Logistic regression
tf.reset_default_graph()

with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="image") # mnist data image of shape 28*28=784

y = tf.placeholder(tf.float32, [None, 10], name= "y_true_class") # 0-9 digits recognition => 10 classes

# Set model weights
with tf.name_scope("classification"):
    w = tf.Variable(tf.zeros([784, 1]), name="weights")
    w_0 = tf.Variable(tf.zeros([1]), name="bias")
    linear = tf.matmul(x, w) + w_0

# Construct model
with tf.name_scope("activation"):
    pred = tf.nn.sigmoid(linear) # Sigmoid

In [4]:
# Minimize error using cross entropy
eta = 0.1
with tf.name_scope("loss"):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(eta).minimize(cost)

In [18]:
show_graph(tf.get_default_graph())

In [10]:
# Two layer neural network
tf.reset_default_graph()

with tf.name_scope("input"):
    x = tf.placeholder(tf.float32, [None, 784], name="image") # mnist data image of shape 28*28=784

y = tf.placeholder(tf.float32, [None, 10], name= "y_true_class") # 0-9 digits recognition => 10 classes

# Hidden layer
with tf.name_scope("hidden"):
    W = tf.Variable(tf.zeros([784, 100]), name="weights")
    w_0 = tf.Variable(tf.zeros([100]), name="bias")
    linear = tf.matmul(x, W) + w_0

# Set model weights
with tf.name_scope("classification"):
    w = tf.Variable(tf.zeros([100, 1]), name="weights")
    w_0 = tf.Variable(tf.zeros([1]), name="bias")
    linear = tf.matmul(linear, w) + w_0

# Construct model
with tf.name_scope("activation"):
    pred = tf.nn.sigmoid(linear) # Sigmoid

In [11]:
# Minimize error using cross entropy
with tf.name_scope("loss"):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(cost)

In [12]:
show_graph(tf.get_default_graph())

In [17]:


# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(10):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/10)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(10)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step


    print("Optimization Finished!")


Optimization Finished!
